# Challenge
Clean data with Pandas

## Details:
Use the provided JSON dataset and complete the following:
1. Some string values may be, "" instead of null. Set these fields to NaN with numpy before you continue cleaning
2. Clamp publication year to the years of 1800-2026
3. Ensure average_rating, page_count, etc can't have impossible values
4. Normalize dates in "last_checkout"
5. Ensure genres, languages, formats, and publishers have proper capitalization
6. Find and remove duplicate books with the same title/author
7. Drop any rows with empty fields

In [5]:
import pandas as pd
import numpy as np
import re

df = pd.read_json("../books_dirty.json")

# 1. replace empty strings with nan
df.replace("", np.nan, inplace=True)

# 3. ensure numeric values can't have impossible values
df["publication_year"] = pd.to_numeric(df["publication_year"], errors="coerce")
df["publication_year"] = df["publication_year"].fillna(0)

df["average_rating"] = pd.to_numeric(df["average_rating"], errors="coerce")
df["average_rating"] = df["average_rating"].fillna(0)
df["average_rating"] = df["average_rating"].clip(1, 5)

df["page_count"] = pd.to_numeric(df["page_count"], errors="coerce")
df["page_count"] = df["page_count"].fillna(0)
df["page_count"] = df["page_count"].clip(120, 1100)

df["ratings_count"] = pd.to_numeric(df["ratings_count"], errors="coerce")
df["ratings_count"] = df["ratings_count"].fillna(0)
df["ratings_count"] = df["ratings_count"].clip(25, 1000)

df["price_usd"] = pd.to_numeric(df["price_usd"], errors="coerce")
df["price_usd"] = df["price_usd"].fillna(0)
df["price_usd"] = df["price_usd"].clip(7.99, 14.99)

df["sales_millions"] = pd.to_numeric(df["sales_millions"], errors="coerce")
df["sales_millions"] = df["sales_millions"].fillna(0)
df["sales_millions"] = df["sales_millions"].clip(0.01, 15)

# 4. normalize last_checkout dates
df["last_checkout"] = df["last_checkout"].replace("N/A", np.nan)
df["last_checkout"] = pd.to_datetime(df["last_checkout"], format="mixed")

# 5. ensure genres, languages, formats, and publishers have proper capitalization (.title())
df["genre"] = df["genre"].str.title()
df["language"] = df["language"].str.title()
df["language"] = df["language"].replace("Eng", "English")
df["format"] = df["format"].str.title()
df["publisher"] = df["publisher"].str.title()

# 2. clamp publication year to 1800 - 2026
df["publication_year"] = df["publication_year"].fillna(0)
df["publication_year"] = df["publication_year"].clip(1800, 2026)

# 6. find and remove duplicate books with same title/author
df.drop_duplicates(subset=["title", "author"], inplace=True)

# 7. drop rows with empty fields
df.dropna(inplace=True)

# additional: regex emails
email_regex = r"\.[\w]+$"
df["publisher_email"] = df["publisher_email"].str.replace(email_regex, ".com", regex=True)

df



,book_id,title,author,genre,publication_year,page_count,average_rating,ratings_count,price_usd,publisher,language,format,in_print,sales_millions,last_checkout,available,publisher_email
7,83e1fcbf-b602-4f7d-954c-0c4261252d33,Book Title 19,Author 24,History,1800.0,120.0,4.89,25.0,14.99,North Star Press,French,Paperback,True,0.01,2024-08-07 13:10:33.617058,false,eu.contact@oldtreepublishing.com
15,7b1a7a3f-e665-4e88-bae9-797804c808fe,Book Title 16,Author 15,Science Fiction,1800.0,120.0,1.00,1000.0,7.99,Sunshine Media,English,Ebook,true,0.01,2025-09-09 13:10:33.617292,False,press.galacticbooks@galacticbooks.com
36,c3405e73-67ae-4174-8a8c-3002bab0f6c7,Book Title 20,Author 29,Science Fiction,1800.0,120.0,1.00,25.0,7.99,Galactic Books,German,Audio Book,true,15.00,2025-02-07 13:10:33.617875,True,support@galacticbooks.com
54,ed0ed6fa-3483-4b70-a757-b0728a8e34d1,Book Title 18,Author 21,History,1925.0,120.0,1.00,25.0,14.99,Galactic Books,German,Audiobook,False,11.15,2024-02-18 13:10:33.618428,true,mx.galacticbooksinfo@support.galacticbooks6.com
86,6eaa897f-bace-484f-ad9d-91b0ba0af52f,Book Title 1,Author 28,Biography,1872.0,579.0,1.00,25.0,7.99,North Star Press,English,Paperback,False,0.01,2023-02-27 13:10:33.619518,True,lumen94@mx.northstarpressbooks.core.com
89,801eb47d-e204-4201-91a8-0afa025fbbd5,Book Title 9,Author 12,Biography,1800.0,120.0,5.00,1000.0,7.99,Old Tree Publishing,English,Paperback,true,0.01,2023-11-23 13:10:33.619590,True,kr.jwright@www.northstarpress.com
94,5155c2b3-affe-47ee-81ff-9c10c37b0bbc,Book Title 3,Author 17,Romance,1800.0,120.0,1.00,25.0,7.99,North Star Press,English,Paperback,true,0.01,2024-04-14 13:10:33.619724,True,kr.johnlee@redrockpress.com
103,9ad027f7-ad6d-4bba-a367-d180d9900cab,Book Title 12,Author 29,Technology,1800.0,120.0,4.78,835.0,7.99,Old Tree Publishing,English,Ebook,false,0.01,2025-09-27 13:10:33.619968,True,sales@blueoak.com
104,2f6cafe6-63e3-4fd0-a39c-84157fba7c9a,Book Title 2,Author 3,Fantasy,1800.0,120.0,5.00,1000.0,14.99,Galactic Books,German,Ebook,false,0.01,2023-10-11 13:10:33.620002,false,mx.sky8@mail.galacticbooks.com
105,fca511b5-e19d-4cdc-86ce-ae77ea2dbe54,Book Title 4,Author 16,Romance,1800.0,120.0,1.00,25.0,7.99,Old Tree Publishing,French,Hardcover,False,0.01,2024-11-15 13:10:33.620041,false,sales@oldtreepublishing.alpha.com
